In [1]:
import pandas as pd
from pandas import TimeSeries
import numpy as np
import math
from bokeh.charts import TimeSeries, output_file, show
import statsmodels.tsa.stattools as stat

In [13]:
sp500=pd.read_csv('C:/mine/Course/9894/project/data/sp_500_daily_prices.csv',usecols=['date','COMNAM','PRC',\
                    'RET','ASK','BID'],engine='c')

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [16]:
sp_recent=sp500[sp500['date']>=start]
sp_recent=sp_recent[sp_recent['date']<=end]

In [24]:
start+int(0.9*(end-start))

20145101

In [26]:
data_in=sp_recent[sp_recent['date']>=start+int(0.9*(end-start))]
data_in=data_in[data_in['date']<=end]
data_in

,date,COMNAM,PRC,RET,BID,ASK


In [30]:
start=20130101
end=20150101

In [31]:
data,pairs=find_pair(start,end)

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2881: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\ProgramData\Anaconda3\lib\site-packages\numpy\linalg\linalg.py:1546: RuntimeWarning: invalid value encountered in greater
  return sum(S > tol)
C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:875: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:875: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1731: RuntimeWarning: invalid value encountered in greater_equal
  cond2 = (x >= self.b) & cond0


In [32]:
data

,date,COMNAM,PRC,RET,BID,ASK
12956,20141020,ORACLE CORP,37.80,-0.001848,37.79,37.80
12957,20141021,ORACLE CORP,38.35,0.01455,38.34,38.35
12958,20141022,ORACLE CORP,37.64,-0.018514,37.64,37.65
12959,20141023,ORACLE CORP,38.23,0.015675,38.22,38.23
12960,20141024,ORACLE CORP,38.73,0.013079,38.72,38.73
12961,20141027,ORACLE CORP,38.43,-0.007746,38.42,38.43
12962,20141028,ORACLE CORP,38.65,0.005725,38.62,38.63
12963,20141029,ORACLE CORP,38.58,-0.001811,38.58,38.59
12964,20141030,ORACLE CORP,38.50,-0.002074,38.50,38.51
12965,20141031,ORACLE CORP,39.05,0.014286,39.01,39.02


In [27]:
def data_process(start,end):
    sp500=pd.read_csv('C:/mine/Course/9894/project/data/sp_500_daily_prices.csv',usecols=['date','COMNAM','PRC',\
                    'RET','ASK','BID'],engine='c')
    sp_recent=sp500[sp500['date']>=start]
    sp_recent=sp_recent[sp_recent['date']<=end]
    
    date_list=sorted(list(sp_recent['date']))
    company_recent=list(sp_recent[sp_recent['date']==date_list[0]]['COMNAM'])
    sp_orig=sp_recent[sp_recent['COMNAM']==company_recent[0]][['date','RET']]
    sp_orig=pd.DataFrame(list(sp_orig['RET']),columns=[company_recent[0]],index=list(sp_orig['date']))
    sp_orig=sp_orig[~sp_orig.index.duplicated(keep='first')]
    for i in range(1,len(company_recent)):
        company=company_recent[i]
        current_data=sp_recent[sp_recent['COMNAM']==company][['date','RET']]
        ret=pd.DataFrame(list(current_data['RET']),columns=[company],index=list(current_data['date']))
        ret =ret[~ret.index.duplicated(keep='first')]
        sp_orig=pd.concat([sp_orig,ret],join='outer',axis=1)
    sp_ret=sp_orig.loc[date_list[0]:date_list[int(0.9*len(date_list))],:]
    sp_ret=sp_ret.dropna(axis=1)
    
    
    sp_price=sp_recent[sp_recent['COMNAM']==company_recent[-1]][['date','PRC']]
    sp_price=pd.DataFrame(list(sp_price['PRC']),columns=[company_recent[-1]],index=list(sp_price['date']))
    sp_price=sp_price[~sp_price.index.duplicated(keep='first')]
    for company in company_recent:
        current_data=sp_recent[sp_recent['COMNAM']==company][['date','PRC']]
        price=pd.DataFrame(list(current_data['PRC']),columns=[company],index=list(current_data['date']))
        price =price[~price.index.duplicated(keep='first')]
        sp_price=pd.concat([sp_price,price],join='outer',axis=1)
    sp_price=sp_price.loc[date_list[0]:date_list[int(0.9*len(date_list))],:]
    
    return sp500,sp_ret,sp_price,date_list

In [28]:
def filter_pair(pairs):
    selected_pair=[]
    stock_list=[]
    for pair in pairs:
        if pair[0] not in stock_list and pair[1] not in stock_list:
            selected_pair+=[pair]
            stock_list+=pair
    return selected_pair

In [29]:
def find_pair(start,end):
    sp500,sp_ret,sp_price,date_list=data_process(start,end)
    error_return=['E','D','C','B','A','']
    sp_ret=sp_ret.applymap(lambda x:np.float64(0.0) if x in error_return else np.float64(x))
    sp_pca=np.array(sp_ret)
    cov_company=np.cov(sp_pca)
    cov_company[np.isnan(cov_company)] = 0.0
    eigen_val,eigen_vector=np.linalg.eig(cov_company)
    eigen_vector_max=list(zip(eigen_vector[np.argmax(eigen_val)],sp_ret.columns))
    eigen_vector_max.sort(key=lambda x:x[0],reverse=True)
    eigen_vector_max=list(zip(*eigen_vector_max))
    data=sp_price[list(eigen_vector_max[1][0:20])]
    score_matrix, pvalue_matrix, pairs=find_cointegrated_pairs(data)
    selected_pair=filter_pair(pairs)
    data_inuse=sp_recent[sp_recent['date']>=date_list[int(0.9*len(date_list))]]
    data_inuse=data_inuse[data_inuse['date']<=end]
    return data_inuse,selected_pair

In [7]:
def find_cointegrated_pairs(data):
    n = len(data.columns)
    score_matrix = np.zeros((n, n))
    pvalue_matrix = np.ones((n, n))
    keys = list(data.columns)
    pairs = []
    for i in range(n):
        for j in range(i+1, n):
            S1 = np.array(data[keys[i]])
            S2 = np.array(data[keys[j]])
            result = stat.coint(S1, S2)
            score = result[0]
            pvalue = result[1]
            score_matrix[i, j] = score
            pvalue_matrix[i, j] = pvalue
            if pvalue < 0.01:
                pairs.append((keys[i], keys[j]))
    return score_matrix, pvalue_matrix, pairs

In [13]:
sp_new=sp_new.dropna(axis=1)

In [14]:
error_return=['E','D','C','B','A','']

In [15]:
sp_new=sp_new.applymap(lambda x:np.float64(0.0) if x in error_return else np.float64(x))

In [16]:
sp_pca=np.array(sp_new)

In [17]:
cov_company=np.cov(sp_pca)

In [18]:
cov_company[np.isnan(cov_company)] = 0.0

In [19]:
eigen_val,eigen_vector=np.linalg.eig(cov_company)

In [20]:
eigen_vector_max=list(zip(eigen_vector[np.argmax(eigen_val)],sp_new.columns))

In [21]:
eigen_vector_max.sort(key=lambda x:x[0],reverse=True)

In [22]:
eigen_vector_max=list(zip(*eigen_vector_max))

In [25]:

#sp_price=sp_price.dropna(axis=1)

In [27]:
data=sp_price[list(eigen_vector_max[1][:20])]

In [28]:
score_matrix, pvalue_matrix, pairs=find_cointegrated_pairs(data)

In [43]:
selected_pair=filter_pair(pairs)

In [ ]:
selected_pair=filter_pair(pairs)
data_inuse=sp_recent=sp500[sp500['date']>=date_list[int(0.9*len(date_list))] and sp500['date']<=date_list[-1]]

In [8]:
sp_orig=sp_recent[sp_recent['COMNAM']==company_recent[0]][['date','RET']]
sp_orig=pd.DataFrame(list(sp_orig['RET']),columns=[company_recent[0]],index=list(sp_orig['date']))
sp_orig=sp_orig[~sp_orig.index.duplicated(keep='first')]